In [ ]:
import arcgis
import getpass
import logging
import pandas as pd
from palletjack import extract, transform, load


In [ ]:
#: Set up logging
logging.basicConfig(level=logging.DEBUG)

#: Get our GIS object
username = getpass.getpass(prompt="Username: ")
gis = arcgis.gis.GIS("https://www.arcgis.com", username)

In [ ]:
#: Extract spreadsheet data
sheet_loader = extract.GSheetLoader(r'c:\gis\git\pj-uorg\src\uorg\sheets-sa.json')
new_data = sheet_loader.load_specific_worksheet_into_dataframe("1LbHWR0_wADXdw-10nXO8HkBC62-5rgTlscv8mzTnhmE", 0)
new_data.head()

In [ ]:
#: Compute the favorite game, fix the county names
new_data["favorite"] = new_data[["Legend of Zelda", "Ocarina of Time", "Breath of the Wild"]].idxmax(axis=1)
new_data["County"] = new_data["County"].str.replace("County", "").str.strip().str.upper()
new_data.head()

In [ ]:
#: Load the existing Data
feature_layer = arcgis.features.FeatureLayer.fromitem(gis.content.get("75ca1488109d4286b16b5df00d5cc70c"))
live_df = feature_layer.query(as_df=True)

In [ ]:
#: Update the favorite game based on the new data
live_df["favorite_zelda"] = live_df.merge(new_data[["County", "favorite"]], left_on="name", right_on="County")["favorite"]
live_df.head()

In [ ]:
#: Load our new date into the Feature Service
loader = load.FeatureServiceUpdater(gis, "75ca1488109d4286b16b5df00d5cc70c", r"c:\temp")
features_updated = loader.update_features(live_df)

In [ ]:
features_updated